## Homework 1
#### DS7337: Natural Language Processing, Fall 2022
By: Allen Hoskins

***
## 1. Install Python and NLTK

In [1]:
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from nltk import text
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
#### Method to get the Vocab Size and normalize the score

def n_vocab_size(*arg):
    vocab_size = np.array([])
    vocab_size_norm = np.array([])
    
    #### Getting the Vocab Size
    for text in arg:
        vocab_size = np.append(vocab_size,len(set(text)))
    
    #### Normalizing using the formula 
    for vsize in vocab_size:
        vocab_size_norm = np.append(vocab_size_norm,(vsize - vocab_size.min()) /
                                                    (vocab_size.max() - vocab_size.min()))
    
    #### Normalizing using sklearn preprocessing 
    vocab_size_norm_sklearn = MinMaxScaler(vocab_size,feature_range=(0,1))
    
    return(vocab_size,vocab_size_norm,vocab_size_norm_sklearn)



In [3]:
HW2_TEXT = gutenberg.words('austen-emma.txt')

In [11]:
def url_text(url):
    req = requests.get(url).text
    resp = nltk.word_tokenize(req)
    return nltk.Text(resp)

def long_words(text,is_url=False):
    if is_url == True:
        unique_text = set(url_text(text))
        long_text = [w for w in unique_text if len(w)> 10]
    elif is_url==False:
        unique_text = set(text)
        long_text = [w for w in unique_text if len(w)> 10]
    return sorted(long_text)

In [7]:
HW1_URL = 'https://www.gutenberg.org/cache/epub/22925/pg22925.txt'

In [13]:
url_text(HW1_URL)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/allen/nltk_data'
    - '/Users/allen/virtualenvs/NLP/nltk_data'
    - '/Users/allen/virtualenvs/NLP/share/nltk_data'
    - '/Users/allen/virtualenvs/NLP/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
long_words(HW2_URL,is_url=True)

In [ ]:
long_words(HW2_TEXT,is_url=False)

In [ ]:
long_words(url,flag=True)

In [ ]:
vocab_size

In [ ]:
print("Normalized Values by using sklearn", *vocab_size[2],sep='\n- ')